In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
model = ItemKNNCFRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2573 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [10]:
import optuna as op
def objective(trial):
    topK = trial.suggest_int("topK", 2, 1000)
    shrink = trial.suggest_int("shrink", 2, 1000)
    similarity = trial.suggest_categorical("similarity", ["tversky"])
    normalize = trial.suggest_categorical("normalize", [True, False])
    feature_weighting = trial.suggest_categorical("feature_weighting", ["none", "BM25", "TF-IDF"])
    tversky_alpha = trial.suggest_uniform("tversky_alpha", 0.0, 2.0)
    tversky_beta = trial.suggest_uniform("tversky_beta", 0.0, 2.0)
    #similarity_args = {"t_a": tversky_alpha, "t_b": tversky_beta}
    
    recommender = model(URM_train)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [11]:
best_params = {
    "topK": 22,
    "shrink": 6431,
    "similarity": "tversky",
    "normalize": True,
    "feature_weighting": "BM25",
    "tversky_alpha": 0.0,
    "tversky_beta": 0.0
}
# {"topK": 9, "shrink": 124, "similarity": "cosine", "normalize": true, "feature_weighting": "TF-IDF"}
# {'topK': 22, 'shrink': 6431, 'similarity': 'tversky', 'normalize': True, 'feature_weighting': 'BM25'}
# {'topK': 24, 'shrink': 303, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}
study = op.create_study(direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-11-27 15:50:13,356] A new study created in memory with name: no-name-f6f55615-1307-4cd1-a73e-f3d2d8e86383
/tmp/ipykernel_16881/1486519147.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  tversky_alpha = trial.suggest_uniform("tversky_alpha", 0.0, 2.0)
/tmp/ipykernel_16881/1486519147.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  tversky_beta = trial.suggest_uniform("tversky_beta", 0.0, 2.0)


ItemKNNCFRecommender: URM Detected 459 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 322 ( 1.4%) items with no interactions.
Similarity column 22222 (100.0%), 9057.14 column/sec. Elapsed time 2.45 sec
EvaluatorHoldout: Processed 10065 (100.0%) in 6.09 sec. Users per second: 1651


[I 2023-11-27 15:50:21,980] Trial 0 finished with value: 0.02803554278144449 and parameters: {'topK': 22, 'shrink': 6431, 'similarity': 'tversky', 'normalize': True, 'feature_weighting': 'BM25', 'tversky_alpha': 0.0, 'tversky_beta': 0.0}. Best is trial 0 with value: 0.02803554278144449.


ItemKNNCFRecommender: URM Detected 459 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 322 ( 1.4%) items with no interactions.
Similarity column 22222 (100.0%), 8638.85 column/sec. Elapsed time 2.57 sec
EvaluatorHoldout: Processed 10065 (100.0%) in 7.83 sec. Users per second: 1286


[I 2023-11-27 15:50:32,597] Trial 1 finished with value: 0.021875046325866055 and parameters: {'topK': 478, 'shrink': 308, 'similarity': 'tversky', 'normalize': True, 'feature_weighting': 'TF-IDF', 'tversky_alpha': 1.0957822828477461, 'tversky_beta': 0.9383021209819413}. Best is trial 0 with value: 0.02803554278144449.


...

[I 2023-11-27 16:05:13,866] Trial 97 finished with value: 0.02811651250995524 and parameters: {'topK': 58, 'shrink': 439, 'similarity': 'tversky', 'normalize': False, 'feature_weighting': 'BM25', 'tversky_alpha': 0.24300643758848672, 'tversky_beta': 0.5049929828352445}. Best is trial 95 with value: 0.029093385060598324.


ItemKNNCFRecommender: URM Detected 459 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 322 ( 1.4%) items with no interactions.
Similarity column 22222 (100.0%), 8661.51 column/sec. Elapsed time 2.57 sec
EvaluatorHoldout: Processed 10065 (100.0%) in 8.35 sec. Users per second: 1205


[I 2023-11-27 16:05:25,109] Trial 98 finished with value: 0.02224309448899615 and parameters: {'topK': 804, 'shrink': 610, 'similarity': 'tversky', 'normalize': True, 'feature_weighting': 'BM25', 'tversky_alpha': 0.6039027333542167, 'tversky_beta': 0.5972339138465501}. Best is trial 95 with value: 0.029093385060598324.


ItemKNNCFRecommender: URM Detected 459 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 322 ( 1.4%) items with no interactions.
Similarity column 22222 (100.0%), 8879.66 column/sec. Elapsed time 2.50 sec
EvaluatorHoldout: Processed 10065 (100.0%) in 6.67 sec. Users per second: 1509


[I 2023-11-27 16:05:34,437] Trial 99 finished with value: 0.026665235493104424 and parameters: {'topK': 152, 'shrink': 315, 'similarity': 'tversky', 'normalize': True, 'feature_weighting': 'BM25', 'tversky_alpha': 0.38341757119619446, 'tversky_beta': 0.6444771861317137}. Best is trial 95 with value: 0.029093385060598324.


In [12]:
study.best_params

{'topK': 31,
 'shrink': 435,
 'similarity': 'tversky',
 'normalize': True,
 'feature_weighting': 'BM25',
 'tversky_alpha': 0.17113169506422393,
 'tversky_beta': 0.5684024974085575}

In [13]:
final = model(URM_train_validation)
final.fit(**study.best_params)

ItemKNNCFRecommender: URM Detected 209 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 107 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 8332.19 column/sec. Elapsed time 2.67 sec


In [14]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2240 (17.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10398 (100.0%) in 6.11 sec. Users per second: 1703


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.087555                 0.150164  0.126038  0.044167    0.075071   
 
              MRR      NDCG       F1 HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                      ...                 
 10      0.250032  0.137193  0.10333  0.49346      0.328143  ...      0.822757   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.405998    0.822757       0.065133       10.416567   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.998268             0.187055              0.802518   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cut

In [ ]:
MAP = .0446210

In [15]:
boh = ItemKNNCFRecommender(URM_train_validation)
boh.fit(topK=22, shrink=6431, similarity="tversky", normalize=True, feature_weighting="BM25")
evaluator_test.evaluateRecommender(boh)

ItemKNNCFRecommender: URM Detected 209 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 107 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 8086.40 column/sec. Elapsed time 2.75 sec
EvaluatorHoldout: Processed 10398 (100.0%) in 6.34 sec. Users per second: 1641


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.086103                 0.146907  0.122983  0.043788    0.073726   
 
              MRR      NDCG       F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.250445  0.135174  0.10129  0.489421      0.326844  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.822757          0.402674    0.822757       0.057244   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            9.904492                   0.996415             0.164398   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      